In [ ]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

## Stdlib
from collections import Counter, defaultdict

## Non-std libs
from matplotlib.colors import hsv_to_rgb
import matplotlib.pyplot as plt

## Local modules
from scn_rrd import rrd_meta_utils, rrd_utils, plot_utils

References
- Unit is millisec.
    - https://github.com/librenms/librenms/blob/master/LibreNMS/Data/Source/FpingResponse.php :: `__construct()`
    - https://github.com/librenms/librenms/blob/fb3c2ecb1c9275108058e76e8af6f145a92235e7/LibreNMS/Polling/ConnectivityHelper.php :: `savePingStats()`

In [ ]:
locations = [
    'FCS',
    'lihi-southend',
    'nickelsville-cd',
    'northlake',
    'occ-oromo',
    'sps-franklin',
    'sps-garfield',
]

meta = rrd_meta_utils.read_meta()
meta = meta[ meta['location'].isin(locations) ]
devices_info = meta[['hostname', 'location']].drop_duplicates()
hostname_to_phys_loc = {
    hostname : phys_loc
    for (_, (hostname, phys_loc)) in devices_info.iterrows()
}

logical_loc_to_hostnames = defaultdict(list)
for (hostname, phys_loc) in hostname_to_phys_loc.items():
    logical_loc = plot_utils.PHYS_LOC_TO_LOGICAL_LOC[phys_loc]
    logical_loc_to_hostnames[logical_loc].append(hostname)

In [ ]:
hostname_to_df = rrd_utils.read_rrds(
    hostname_to_phys_loc.keys(),
    rrd_meta_utils.DEVICE_AGNOSTIC_RRD_FILENAMES.latency,
    '-20day',
)

In [ ]:
(fig, ax) = plt.subplots(figsize=(12, 6))
for (logical_loc, hostnames) in logical_loc_to_hostnames.items():
    for (i, hostname) in enumerate(hostnames):
        df = hostname_to_df[hostname]
        
        color_h = plot_utils.LOC_TO_COLOR_H[logical_loc]
        color_s = (i+1) / (len(hostnames)+1)
        
        ax.plot(df['time'], df['ping'], label=hostname, color=hsv_to_rgb((color_h, color_s, 1)))
ax.set_title('Latency')
ax.set_ylabel('millisec')
ax.legend()
None # Hide stdout output of the above line